In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset_dir = '/content/drive/MyDrive/IndoorOutdoorDataset'

In [4]:
img_width, img_height = 224, 224
batch_size = 32
epochs = 100

In [5]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    seed=42,
    shuffle=True,
    subset='training',
)

Found 800 files belonging to 2 classes.
Using 640 files for training.


In [6]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    seed=42,
    shuffle=True,
    subset='validation',
)

Found 800 files belonging to 2 classes.
Using 160 files for validation.


In [7]:
model = models.Sequential()
model.add(ResNet50V2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3), classes=2))
model.add(layers.Flatten())
model.add(layers.Dense(264, activation='sigmoid'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='sigmoid'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation='sigmoid'))

94668760/94668760 [==============================] - 0s 0us/step


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 264)               26493192  
                                                                 
 dropout (Dropout)           (None, 264)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               33920     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 1

In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds
)

Epoch 1/100
20/20 [==============================] - 104s 2s/step - loss: 0.4631 - accuracy: 0.8016 - val_loss: 1.1245 - val_accuracy: 0.4938
Epoch 2/100
20/20 [==============================] - 8s 325ms/step - loss: 0.3083 - accuracy: 0.8828 - val_loss: 0.8528 - val_accuracy: 0.5063
Epoch 3/100
20/20 [==============================] - 8s 337ms/step - loss: 0.2786 - accuracy: 0.9094 - val_loss: 0.7720 - val_accuracy: 0.6000
Epoch 4/100
20/20 [==============================] - 7s 317ms/step - loss: 0.2847 - accuracy: 0.9000 - val_loss: 0.7110 - val_accuracy: 0.5063
Epoch 5/100
20/20 [==============================] - 7s 330ms/step - loss: 0.2729 - accuracy: 0.9016 - val_loss: 1.1342 - val_accuracy: 0.5000
Epoch 6/100
20/20 [==============================] - 7s 317ms/step - loss: 0.2572 - accuracy: 0.9109 - val_loss: 1.2559 - val_accuracy: 0.5063
Epoch 7/100
20/20 [==============================] - 7s 328ms/step - loss: 0.2230 - accuracy: 0.9203 - val_loss: 1.0337 - val_accuracy: 0.6375


In [11]:
model.save('/content/drive/MyDrive/resnet50v2dropout_indoor_outdoor_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [43]:
model.evaluate(val_ds)

5/5 [==============================] - 1s 102ms/step - loss: 0.3174 - accuracy: 0.9250


[0.3174486458301544, 0.925000011920929]